In [1]:
# import package
from datetime import datetime as dtime
from datetime import date,timedelta
from urllib.request import urlopen
import requests
from dateutil import rrule
import matplotlib.pyplot as plt
from datetime import datetime
import pandas as pd
import numpy as np
import json
import time
import os
from io import StringIO
from Tools import *
import csv
from Tools import *
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

In [2]:
# 讀取INI檔案
InitFile = IniTool()
# 讀取路徑
DataPath = InitFile.Read('Infomation', 'DataPath')
# 總資料開始日期
DataStartDate = InitFile.Read('Infomation', 'DataStartDate')
# 開始統計的日期
StartDate = InitFile.Read('Infomation', 'StartDate')


Load init file: .\Initialization.ini


In [3]:
def GetDataFrame_(date):
    url = 'https://www.twse.com.tw/exchangeReport/MI_INDEX?response=csv&date=' + date + '&type=ALL';
    print (url)
    # 下載股價
    r = requests.post('https://www.twse.com.tw/exchangeReport/MI_INDEX?response=csv&date=' + date + '&type=ALL')
    # 整理一些字串：
#     df = df.apply(lambda s: pd.to_numeric(s.astype(str).str.replace(",", "").replace("+", "1").replace("-", "-1"), errors='coerce'))
    # 整理資料，變成表格
    df = pd.read_csv(StringIO(r.text.replace("=", "")), 
            header=["證券代號" in l for l in r.text.split("\n")].index(True)-1)
    return df

In [4]:
def GetDataFrame(date):
    file = DataPath + date + '.csv'
    datas = pd.DataFrame()
#     print(file)
#     with open(file) as csvFile:
#         csvReader = csv.reader(csvFile)
#         datas = list(csvReader)
        
    f = open(file, 'r')
    text = f.read()
    f.close()
#     for line in f.readlines():
#         print(line)

    
#     print(datas[0])
    
    df = pd.read_csv(StringIO(text.replace("=", "")), thousands=",",
        header=["證券代號" in l for l in text.split("\n")].index(True)-1)
    df.insert(0,'Date',date)
#     new_header = datas.loc['證券代號']
#     new_header = create_new_header(new_header)
#     df = datas.loc['證券代號']
    return df
# GetDataFrame('20210804')

In [ ]:
#GetDataFrame('20210804')

In [1]:
def GetDayList(begin, end):
    # 建立搜索清單
    DayList = []
    for i in range((end - begin).days+1):
        day = begin + timedelta(days=i)
        w_day = day.weekday()
        c_day = day.strftime("%Y%m%d")
        NotInList = True
        for index, row in df_HS.iterrows():
            if str(c_day) == str(row['Date']):
                NotInList = False
        if (w_day != 5 and w_day !=6 and NotInList):
            DayList.append(c_day)
    return DayList

In [6]:
# 字串轉Datetime
begin = datetime.strptime(StartDate, "%Y-%m-%d")
begin

datetime.datetime(2020, 1, 1, 0, 0)

In [16]:
# end = datetime.today() - timedelta(days=2)
end = datetime.today()
print (end)

2021-09-11 00:19:10.431868


In [17]:
# 讀取休市清單
df_HS = pd.read_csv('holidaySchedule.csv', index_col = 0)
df_HS.loc[:,'Date'].min()

20190101

In [18]:
# 建立搜索清單
DayList = GetDayList(begin, end)


In [19]:
ListLen = len(DayList)
print(DayList[-1])
ListLen

20210910


410

In [20]:
StartTime = dtime.now()
for idx, val in enumerate(DayList):
    CurrentTime = dtime.now()
    if val == CurrentTime.strftime("%Y%m%d") and CurrentTime.hour < 14:
        break;
    
    fullpath = DataPath + val + '.csv'
    # 確認是否有舊資料
    NoHistory = False
    if os.path.isfile(fullpath):
        # 檔案存在的處理
        print("\r 檔案存在。" + str(idx), end="")
    else:
        # 檔案不存在的處理
        print("\n檔案不存在。")
        NoHistory = True
    if NoHistory is True:
        url = 'https://www.twse.com.tw/exchangeReport/MI_INDEX?response=csv&date=' + val + '&type=ALL'
        print ("\r Loading... "+ str(round((idx+1)/ListLen*100,2)) + "% " +val + " ", end="")
        r = requests.get(url, allow_redirects=True)
        open(fullpath, 'wb').write(r.content)
        time.sleep(15)
    


 檔案存在。407
檔案不存在。
 Loading... 99.76% 20210909 
檔案不存在。
 Loading... 100.0% 20210910 

In [ ]:
test1 = GetDataFrame('20210803')
fliter = (test1['證券代號'] == '3034')
test1[fliter]

In [ ]:
# 建立特定股票的清單
List = []
DFList = list()
for idx, val in enumerate(DayList):
    CurrentTime = dtime.now()
    if val == CurrentTime.strftime("%Y%m%d") and CurrentTime.hour < 14:
        break;
    
    fullpath = DataPath + val + '.csv'
    # 確認是否有舊資料
    NoHistory = False
    if os.path.isfile(fullpath):
        # 檔案存在的處理
        print("\r 檔案存在。" + str(idx) + " " + val, end="")
    else:
        # 檔案不存在的處理
        print("\n檔案不存在。")
        NoHistory = True
    if NoHistory is False:
#         print(val)
        df = GetDataFrame(val)
        DFList.append(df)
#         print(df)
#         fliter = (df['證券代號'] == '3034')

In [ ]:
# 更新清單
def UpdateDFList():
    CurrentTime = dtime.now()
    Start = False
    LastDataDay = DFList[-1].loc[0,'Date']
    for idx, val in enumerate(DayList):

        if LastDataDay == val:
            Start = True
            print(idx)
            continue
        if Start is True:
#             print ("WTF")
            if val == CurrentTime.strftime("%Y%m%d") and CurrentTime.hour < 14:
                break;
            fullpath = DataPath + val + '.csv'
            # 確認是否有舊資料
            NoHistory = False
            if os.path.isfile(fullpath):
                # 檔案存在的處理
                print("\r 檔案存在。" + str(idx) + " " + val, end="")
            else:
                # 檔案不存在的處理
                print("\n檔案不存在。")
                NoHistory = True
            if NoHistory is False:
    #             print(val)
                df = GetDataFrame(val)
                DFList.append(df)
        
UpdateDFList()

In [ ]:
DFList[-1]

In [ ]:
SelectDataList = pd.DataFrame()
for dt in DFList:
#     fliter = (dt['證券代號'] == '3034')
    if SelectDataList.empty:
        SelectDataList = dt.loc[dt['證券代號'] == '1235']
    else:
        SelectDataList = pd.concat([SelectDataList, dt.loc[dt['證券代號'] == '1235']], ignore_index=True )
#     print(dt.loc[dt['證券代號'] == '3034'], ignore_index=True)
SelectDataList

In [ ]:
def GetSelectDataList(src, code):
    dst = pd.DataFrame()
    for dt in src:
        if dst.empty:
            dst = dt.loc[dt['證券代號'] == code]
        else:
            dst = pd.concat([dst, dt.loc[dt['證券代號'] == code]], ignore_index=True )
    return dst

In [ ]:
SelectDataList = GetSelectDataList(DFList, '')
SelectDataList

In [ ]:
# 往前查找
def GetPricePrevious(data, index):
    print('GetPricePrevious')
    if index == 0:
        return -1
    else:
        currentIndex = index - 1
        price = data.loc[current_index, '收盤價']
        if price == '--':
            price = GetPricePrevious(data, currentIndex)
        else:
            return price
    
# 往後查找
def GetPriceNext(data, index):
    print('GetPriceNext')
    DataLen = len(data)
    if index + 1 >= DataLen:
        return -1
    else:
        currentIndex = index + 1
        price = data.loc[currentIndex, '收盤價']
        change = data.loc[currentIndex, '漲跌價差']
        new_price = -1
        if price == '--':
            new_price = GetPriceNext(data, currentIndex)
        else:
            new_price = str(float(price) - change)
        return new_price
    
def GetPrice(data, index):
    # 防呆
    if data.loc[index, '收盤價'] != '--':
        return data.loc[index, '收盤價']
    else:
        price = GetPricePrevious(data, index)
        if price == -1:
            # 往前找沒找到的情況往後找
            price = GetPriceNext(data, index)
        return price



In [ ]:
price = GetPriceNext(SelectDataList, 87)
print(price)
print(SelectDataList.loc[87,'收盤價'])
print(SelectDataList.loc[87,'漲跌價差'])
print(SelectDataList.loc[88,'收盤價'])
print(SelectDataList.loc[88,'漲跌價差'])
df = pd.DataFrame()
df = SelectDataList.loc[86]
print(df)
df = pd.concat([df, SelectDataList.loc[87]], ignore_index=False )
df = pd.concat([df, SelectDataList.loc[88]], ignore_index=False )
df

# pPrice = GetPrice(SelectDataList, 87)
# print(SelectDataList.loc[86,'收盤價'])
# print(SelectDataList.loc[87,'漲跌價差'])
# print(pPrice)
# SelectDataList.loc[87,'收盤價'] = str(pPrice)
# print(SelectDataList.loc[87,'收盤價'])

In [ ]:
def CheckList(code, codelist):
    foundit = False
    for c in codelist:
        if c == code:
            foundit = True
            break
    return foundit

def BuildListByDeal( datalist, lower, day):
    CodeList = []
    count = day
    for idx, dayData in enumerate(reversed(DFList)):
        
        check = False
#         print(dayData)
#         for idy, stockData in enumerate(dayData): 
#         print (dayData)
#         print (len(dayData))
        datacount = 0
        datalen = len(dayData)
#        print (count)
        print("\r" + str(count) + ' ', end="")
        for idy in range(datalen) :
            #print(idy)
            deal = int(dayData.loc[idy,'成交筆數'])
            code = dayData.loc[idy,'證券代號']
            #print (code + ' ' + str(deal))
            #deal = stockData.loc['成交筆數']
#             print(stockData)
#             deal = 0
            
            if deal > lower:
                check = True
                if CheckList(code, CodeList) is False:
                    CodeList.append(code)
        count -= 1
        if count == 0:
            break
    return CodeList
CodeList = BuildListByDeal(DFList, 1000, 10)

print(CodeList)

##### for idx, dayData in enumerate(DFList):
    for idy, price in reversed(list(enumerate(dt['收盤價']))):
        if price == '--':
            print(str(idx) + ' ' + str(idy))
            

In [ ]:
len(CodeList)

In [ ]:
# 依據CodeList建立表格
def GetPriceTable(datalist, codelist):
    PriceDFs = []
    Len = len(CodeList)
    for idy, code in enumerate(codelist):
        print("\r" + str(idy) + '/' + str(Len), end="")
        SelectDataList = GetSelectDataList(datalist, code)
        PriceDFs.append(SelectDataList)
#     print(PriceDF)
#     for idx, dayData in enumerate(datalist):
#         print(idx)
#         for idy, code in enumerate(codelist):
#             price = dayData.loc[idy, '收盤價']
# #             print (price)
#             if price == '--':
#                 print(str(idx) + ' ' + str(idy))
#                 SelectDataList = GetSelectDataList(datalist, code)
#                 print(SelectDataList)
#                 price = GetPricePrevious(SelectDataList,idx)
# #             print(price)
#             PriceDF.iloc[idy,idx] = price
# #         print(PriceDF)
    return PriceDFs
    
PriceDFs = GetPriceTable(DFList,CodeList)


In [ ]:
CodeDataPath = "./CodeData/"
for idx, code in enumerate(CodeList):
    path = CodeDataPath + code + ".csv"
    PriceDFs[idx].to_csv(path, encoding='utf_8_sig')
    

In [ ]:
# PriceDFs[0]
def GetMA(datalist, index):
    List = []
    df = datalist[index]
#     print(df.loc[1])
    print(len(df))
    days = [5,10,20,60,200]
    ma = [0,0,0,0,0]
    total = 0
    count = 0;
    for idx in reversed(range(len(df))):
        count += 1;
        price = df.loc[idx,'收盤價']
        total += float(price)
        if count == 5:
            ma[0] = total / count
        if count == 10:
            ma[1] = total / count
        if count == 20:
            ma[2] = total / count
        if count == 60:
            ma[3] = total / count
        if count == 200:
            ma[4] = total / count
    return ma
            
MA = GetMA(PriceDFs, 3)
print (MA)

In [ ]:
# def GetMA(datalist, code, days):


#     List = []
#     count = days
#     dst = pd.DataFrame()
#     for idx, dayData in enumerate(reversed(DFList)):
        
# #         dst = dayData.loc[dayData['證券代號'] == code]
# #         print(dst)
#         for idy in range(len(dayData)):
#             if code == dayData.loc[idy, '證券代號']:
#                 price = dayData.loc[idy,'收盤價']
#                 if price == '--':
#                     SelectDataList = GetSelectDataList(datalist, code)
#                     GetPricePrevious(SelectDataList,code)
#                 print (dayData.loc[idy,'收盤價'])
#         count -= 1
#         if count <= 0:
#             break
#     return dst
# #    for idx, price in enumerate(data['收盤價']):
# #         print(str(idx)+" "+price)
# #        if price == '--':
# #            print(idx)
# #     for dt in data:
# #           if dt['收盤價'].contains('--'):
# #                 List.append(dt)
# Result = GetMA(DFList, '3006', 5)
# Result